**# Importanto o Dataset e verificando o conteúdo**

In [13]:
import sys
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') #carrego stopwrds para filtragem NLP

# Adiciona o caminho da pasta onde o arquivo 'dataset.py' está localizado
# Isso pula a necessidade de mencionar a pasta com hífen no import
caminho_raw = os.path.abspath(os.path.join('..', 'data', 'raw'))
if caminho_raw not in sys.path:
    sys.path.append(caminho_raw)

from dataset import carregar_dados

df = carregar_dados()
print(df.head())

                                           issue_url  \
0  "https://github.com/zhangyuanwei/node-images/i...   
1     "https://github.com/Microsoft/pxt/issues/2543"   
2  "https://github.com/MatisiekPL/Czekolada/issue...   
3  "https://github.com/MatisiekPL/Czekolada/issue...   
4  "https://github.com/MatisiekPL/Czekolada/issue...   

                                         issue_title  \
0  can't load the addon. issue to: https://github...   
1  hcl accessibility a11yblocking a11ymas mas4.2....   
2  issue 1265: issue 1264: issue 1261: issue 1260...   
3  issue 1266: issue 1263: issue 1262: issue 1259...   
4  issue 1288: issue 1285: issue 1284: issue 1281...   

                                                body  
0  can't load the addon. issue to: https://github...  
1  user experience: user who depends on screen re...  
2  ┆attachments: <a href= https:& x2f;& x2f;githu...  
3  gitlo = github x trello\n---\nthis board is no...  
4  ┆attachments: <a href= https:& x2f;& x2f;githu..

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Processamento e limpeza dos dados**


In [14]:
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'raw')))
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'processed')))


from dataset import verifica_vazios, verifica_frequentes
from data_processing import clean_text

df["clean_title"] = df["issue_title"].apply(clean_text)
df["clean_body"] = df["body"].apply(clean_text)


media_palavras_url = df["issue_url"].str.split().str.len().mean()
media_palavras_title = df["clean_title"].str.split().str.len().mean()
media_palavras_body = df["clean_body"].str.split().str.len().mean()

print(f"Média de palavras em url: {media_palavras_url}, em title: {media_palavras_title}, em body: {media_palavras_body}")

url_vazio = verifica_vazios(df,"issue_url")
titles_vazio = verifica_vazios(df,"clean_title")
body_vazio = verifica_vazios(df,"clean_body")

palavras_frequentes_titles = verifica_frequentes(df,"clean_title", 20)
print(f"Palavras mais frequentes em titles: {palavras_frequentes_titles}")
palavras_frequentes_body = verifica_frequentes(df,"clean_body", 20)
print(f"Palavras mais frequentes em body: {palavras_frequentes_body}")



Média de palavras em url: 1.0, em title: 10.9025, em body: 38.63875
Resultados para 'issue_url':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 0
Resultados para 'clean_title':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 69
Resultados para 'clean_body':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 4
Palavras mais frequentes em titles: [('issu', 352), ('add', 35), ('error', 21), ('bug', 21), ('use', 18), ('set', 12), ('test', 10), ('line', 9), ('task', 9), ('jquery', 8), ('api', 8), ('fix', 6), ('training', 6), ('network', 6), ('app', 5), ('help', 5), ('npm', 5), ('load', 4), ('button', 4), ('warning', 4)]
Palavras mais frequentes em body: [('close', 1237), ('add', 939), ('issues', 898), ('columns', 704), ('list', 540), ('comment', 538), ('column', 531), ('trello', 528), ('update', 367), ('via', 360), ('default', 360), ('move', 358), ('settings', 357), ('custom', 355), ('attachments', 352), ('matisiekpl', 352), ('czekolada', 352), ('gitlo', 352), ('board', 352), ('sync', 3


   **Análise descritiva dos dados processados**

- Média de palavras para entender quantos tokens irão ser processados pela LLM
    - Quantidade de células vazias em todas as colunas
        - não teve resultado de células vazias, todas preenchidas
    - Títulos - tamanhos médio de 5,4 palavras
    - Body - Aproximadamente 28 palavras
- Ideal para:
    - Embeddings
    - Chunking Leve
    - RAG Eficiente(baixo custo e boa semântica)

**Criação de Texto final**

In [15]:
from sentence_transformers import SentenceTransformer

#criação de nova coluna para texto final que será direcionado ao embedding
df["final_text"] = (
    "Title: " + df["clean_title"] +
    ". Body: " + df["clean_body"]
)

#carregando o modelo que será usado, modelo rápido e leve para projeto OBS: Uso da CPU pois GPU esta ultrapassada para o modelo
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device="cpu") #usando o paraphrasal para poder perguntar em portugues
df[["clean_title", "clean_body", "final_text"]].head()

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 833.20it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


,clean_title,clean_body,final_text
0,load addon issue error lib libc version glibc ...,load addon issue error lib libc version glibc ...,Title: load addon issue error lib libc version...
1,hcl accessibility yblocking ymas mas hcl makec...,user experience user depends screen reader get...,Title: hcl accessibility yblocking ymas mas hc...
2,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...
3,issue issue issue issue issue issue issue issu...,gitlo github trello board linked update issue ...,Title: issue issue issue issue issue issue iss...
4,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...


**Embedding**

In [16]:
import numpy as np

#listando o texto final em variavel para ser codificada
texts = df["final_text"].tolist()

#realização do embedding pelo modelo escolhido
embeddings = model.encode( 
    texts,
    show_progress_bar= True
)

#criação do array em np 
embeddings = np.array(embeddings)
print(embeddings.shape)

#persistindo os dados em formato npy e csv para nao necessitar de conversao novamente
np.save("embeddings.npy",embeddings)
df.to_csv("issue_processed.csv", index=False)


Batches: 100%|██████████| 25/25 [00:08<00:00,  3.09it/s]

(800, 384)


**Busca Semântica**

- Primeiramente utilizarei busca ingênua para este caso, mais rápido e ideal para projetos pequenos.
    - será utilizado semelhança de cossenos, variância de -1 a 1, sendo 1 o mais próximo.
    - comparação de vetores do embedding, modelo utilizado de dimensão 384
- Após implementado e projeto funconando, irei dar updgrade para utilizar índice vetorial
- Devido a arquitetura proposta relacionado aos tipos de perguntas para o Chatbot, será implementado:
    - Busca semântica
    - Top k
    - Limiar de similiaridade
    Dessa forma será possível a resposta de perguntas analtícas e explicativas, não somente localizadoras.

In [17]:
import pandas as pd #ler diretamente esta célula sem precisar converter novamente os arquivos no embedding
import numpy as np

embeddings = np.load("embeddings.npy")
df = pd.read_csv("issue_processed.csv")

**Exemplo de pergunta e formato para query**

In [33]:
from semantic_search import encode_query, cosine_similiarity_func

pergunta = "Quais são as issues mais comuns?"
query = encode_query(model, [pergunta])
#print(query.shape)

scores = cosine_similiarity_func(query,embeddings)
scores = scores.flatten()
scores_ordenados = np.argsort(scores)
print(scores.shape)
print(scores_ordenados)

#fazendo busca top_k, depois refatorar montando em funcoes definidas
# Atualizar o Readme e usar o topk como parametro para busca ao refatorar “Utilizamos top-k dinâmico para balancear cobertura semântica e precisão.”
top_k = 30
top_indices = scores_ordenados[-top_k:][::-1] #quero buscar os ultimos 20 valores e ordena-los em sequência maior para menor, pois np.arg retorna ordem crescente
print(top_indices)




(800,)
[798 496 508 670 789 523 607 672 584 709 647 588 482 365 737 480 642 382
 381 794 550 738 404 530 704 712 713 383 546 636 577 707 409 655 520 463
 459 488 598 467 618 473 524 398 570 745 678 369 627 371 486 561 386 622
 626 706 470 541 658 679 516 792 769   0 602 640 674 665 551 721 666 582
 555 601 590 406 778 727 576 744 518 357 359 478 400 431 651 500 747 532
 564 770 723 515 765 781 603 513 785 606 635 569 621 517 580 657 566 685
 372 464 370 702 438 641 595 407 690 356 768 556 428 680 471 736 362 498
 729 725 533 493 728 774 648 355 364 683 687 630 505 629 562 506 579 726
 527 746 559 605 714 720 684 358 654 695 477 617 451 531 732 628 631 772
 447 671 660   1 536 384 444 632 439 499 675 762 761 760 759 756 757 758
 763 771 688 686 681 739 643 395 363 710 691 682 542 563 689 458 507 420
 661 667 777 494 634 623 410 694 367 501 697 481 485 663 696 474 716 604
 385 624 717 452 783 460 380 743 402 609 593 529 374 396 547 649 472 730
 422 405 589 775 748 742 437 645 613 432 715

**Implementando Limiar de similaridade**

 - Utilizei um limiar de similaridade cosseno ajustado empiricamente para garantir que apenas documentos semanticamente relevantes sejam utilizados como contexto para a LLM, reduzindo ruído e alucinação.
 - A princípio foi definido padrão = 0.30, após verifiquei de forma empirica os melhores valores para retornar respostas completas sem interferências de dados fora do padrão

In [34]:
#definir o limiar, fazer um loop retornando os melhores índices
limiar = 0.30
lista_final = []

for i in range(len(top_indices)):
    if scores[top_indices[i]] > limiar:
        lista_final.append({"indice" : top_indices[i] ,"score" :scores[top_indices[i]], "text" : df.iloc[top_indices[i]]["final_text"]})

print(lista_final)

[{'indice': np.int64(462), 'score': np.float32(0.41111237), 'text': 'Title: issues. Body: new issue test'}, {'indice': np.int64(415), 'score': np.float32(0.3567969), 'text': 'Title: issue. Body: got issue tissue'}, {'indice': np.int64(787), 'score': np.float32(0.32647505), 'text': 'Title: issue. Body: body new issue'}, {'indice': np.int64(786), 'score': np.float32(0.32647505), 'text': 'Title: issue. Body: body new issue'}, {'indice': np.int64(412), 'score': np.float32(0.32475835), 'text': 'Title: issue. Body: '}, {'indice': np.int64(429), 'score': np.float32(0.30329826), 'text': 'Title: issue. Body: testing issue issue'}]


**Construção do contexto**

- Utilizar dos dados retornados a partir do limiar e construir os textos mais similares para servir de entrada para LLM
- estou retornando em texto todos os documentos analisados que passaram pelo limiar, poderia restringir a quantidade caso o número de tokens a ser utilizado na LLM seja limitado

In [35]:
MAX_CHARS = 800  # por documento

lista_final = sorted(
    lista_final,
    key=lambda x: x["score"],
    reverse=True
)

contexto = ""
limite = 28

if len(lista_final) <= limite:
    for i in range(len(lista_final)):
        text_limited = lista_final[i]["text"][:MAX_CHARS]

        contexto += (
            f"[Contexto {i+1} | "
            f"Similaridade: {lista_final[i]['score']:.03f}]\n"
            f"{text_limited}\n\n"
        )

else:
    for i in range(limite):
        text_limited = lista_final[i]["text"][:MAX_CHARS]

        contexto += (
            f"[Contexto {i+1} | "
            f"Similaridade: {lista_final[i]['score']:.03f}]\n"
            f"{text_limited}\n\n"
        )

print(contexto)

[Contexto 1 | Similaridade: 0.411]
Title: issues. Body: new issue test

[Contexto 2 | Similaridade: 0.357]
Title: issue. Body: got issue tissue

[Contexto 3 | Similaridade: 0.326]
Title: issue. Body: body new issue

[Contexto 4 | Similaridade: 0.326]
Title: issue. Body: body new issue

[Contexto 5 | Similaridade: 0.325]
Title: issue. Body: 

[Contexto 6 | Similaridade: 0.303]
Title: issue. Body: testing issue issue




**Definir tipos de perguntas para ajustar prompt e resposta**

- Dividir as perguntas em 3 tipos
    - (A) Podem ser respondidas claramente pelo modelo. Ex: "quais tipos", "quais problemas", "sobre o que", "resuma"..
    - (B) Podem ser respondidas com ressalvas. Ex: “mais comuns”,“mais registrados”,“principais” ..
    - (C) O Modelo pode alucinar e dar estatísticas erradas, não recomendado para o propósito da IA. Ex: "quantos", "porcentagem", "frequência", "exata quantidade" ..

In [36]:
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'prompts')))
from rag_prompts import classify_question, build_direct_prompt, build_qualitative_prompt, build_out_of_scope_prompt, build_prompt

QUESTION_TYPE_A = "DIRECT"
QUESTION_TYPE_B = "QUALITATIVE" 
QUESTION_TYPE_C = "OUT_OF_SCOPE"

question_type = classify_question(pergunta)
print("Tipo da pergunta:", question_type)


Tipo da pergunta: QUALITATIVE


**Validador de prompts**

- Antes de passar para LLM analisar, o validador irá verificar se a pergunta se enquadra num tópico a ser analisado ou pode ser respondido sem LLM
- Irá atuar como Gatekeeper, reduzindo a quantidade de tokens analisados
- Evita desperdício de Quota da LLM e redução de custo

**Integração da LLM**

- Estou usando Gemini devido a facilidade de obtenção de uma chave gratuita para estudantes
- A LLM irá receber as informações retiradas do Dataset 
- Será utilizado engenharia de prompt para qualificar a análise da LLM
- No projeto será implantado um scan de input para o usuário digitar sua propria chave api gemini e testar o programa


In [37]:
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'llm')))

from context_validator import validate_context, static_fallback, insufficient_context_response
from dotenv import load_dotenv
import os
from rag_prompts import build_prompt
from gemini_client import generate_answer, answer_question


load_dotenv()
chave_api = os.getenv("GOOGLE_API_KEY")
resposta = answer_question(pergunta,contexto)
print(resposta)

As issues mais comuns parecem ser relativas a "new issue" e testes de "issue". Há também uma repetição da palavra "issue" em alguns títulos e corpos de relato.
